# Data Preparation Module

In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

class HandwritingPreprocessor:
    def __init__(self):
        self.img_size = (128, 128)  # (width, height)
        print(f"HandwritingPreprocessor initialized with image size: {self.img_size}")

    def preprocess_image(self, image_path):
        """
        Preprocess handwritten equation image for neural network input
        """
        print(f"\nStarting image preprocessing for: {image_path}")
        
        # Read image in grayscale
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error: Could not load image from {image_path}")
            return None
        print(f"Original image loaded - Shape: {img.shape}, Type: {img.dtype}")

        # Noise reduction (median blur)
        img = cv2.medianBlur(img, 3)
        print(f"Applied median blur filter (kernel size: 3)")

        # Thresholding (binary inverse)
        _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
        print(f"Applied binary threshold (threshold: 127)")
        print(f"   - Unique pixel values after threshold: {np.unique(img)}")

        # Resize to standard size
        original_shape = img.shape
        img = cv2.resize(img, self.img_size, interpolation=cv2.INTER_NEAREST)
        print(f"Resized from {original_shape} to {img.shape}")

        # Normalize pixel values
        img = img.astype('float32') / 255.0
        print(f"Normalized pixel values - Min: {img.min():.3f}, Max: {img.max():.3f}")

        # Reshape for model input (batch, H, W, C)
        final_shape = (1, self.img_size[1], self.img_size[0], 1)
        img = img.reshape(final_shape)
        print(f"Reshaped for neural network input: {final_shape}")
        print(f"Image preprocessing complete!\n")

        return img
    
    def segment_equation(self, image_path):
        """
        Segment handwritten equation into individual symbols
        """
        print(f"\nStarting equation segmentation for: {image_path}")
        
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error: Could not load image for segmentation")
            return []
        print(f"Image loaded for segmentation - Shape: {img.shape}")
        
        _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
        print(f"Applied binary threshold for contour detection")

        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print(f"Found {len(contours)} contours in the image")

        segments = []
        for i, contour in enumerate(contours):
            x, y, w, h = cv2.boundingRect(contour)
            print(f"   Contour {i+1}: Position=({x},{y}), Size=({w}x{h})", end="")
            
            if w > 10 and h > 10:  # Filter small noise
                segment = binary[y:y+h, x:x+w]
                segments.append({
                    'image': segment,
                    'position': (x, y),
                    'size': (w, h)
                })
                print(f" ACCEPTED (symbol candidate)")
            else:
                print(f" REJECTED (too small - noise)")

        # Sort by x-coordinate (left to right)
        segments.sort(key=lambda s: s['position'][0])
        print(f"Segmentation complete! Found {len(segments)} valid segments")
        
        # Show segment order
        if segments:
            print("Segment order (left to right):")
            for i, seg in enumerate(segments):
                x, y = seg['position']
                w, h = seg['size']
                print(f"   Segment {i+1}: x={x}, y={y}, size={w}x{h}")
        print()

        return segments

# Neural Network for Symbol Recognition

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

class MathSymbolCNN:
    def __init__(self):
        self.model = None
        self.symbols = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                       '+', '-', '*', '/', '=', 'x', 'y', '(', ')', '^']
        self.num_classes = len(self.symbols)
        print(f"MathSymbolCNN initialized")
        print(f"   - Symbol classes: {self.num_classes}")
        print(f"   - Symbols: {self.symbols}")
        
    def build_model(self, input_shape=(128, 128, 1)):
        """
        Build CNN architecture for symbol recognition
        """
        print(f"\nBuilding CNN model with input shape: {input_shape}")
        
        model = keras.Sequential([
            # First Convolutional Block
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            
            # Second Convolutional Block
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            
            # Third Convolutional Block
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),
            
            # Dense layers
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        self.model = model
        print(f"CNN model built successfully!")
        print(f"   - Total parameters: {model.count_params():,}")
        print(f"   - Architecture: Conv2D(32) -> Conv2D(64) -> Conv2D(128) -> Dense(512) -> Dense({self.num_classes})")
        
        return model
    
    def train_model(self, X_train, y_train, X_val, y_val, epochs=50):
        """
        Train the symbol recognition model
        """
        print(f"\nStarting model training...")
        print(f"   - Training samples: {len(X_train)}")
        print(f"   - Validation samples: {len(X_val)}")
        print(f"   - Epochs: {epochs}")
        
        if self.model is None:
            self.build_model()
            
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
        ]
        
        print(f"Training with callbacks: EarlyStopping(patience=10), ReduceLROnPlateau")
        
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=32,
            callbacks=callbacks,
            verbose=1
        )
        
        print(f"Training completed!")
        return history
    
    def predict_symbol(self, image):
        """
        Predict symbol from preprocessed image
        """
        if self.model is None:
            print("Error: Model not trained. Call train_model() first.")
            raise ValueError("Model not trained. Call train_model() first.")
            
        print(f"Predicting symbol from image shape: {image.shape}")
        
        # Make prediction
        prediction = self.model.predict(image, verbose=0)
        symbol_idx = np.argmax(prediction)
        confidence = np.max(prediction)
        predicted_symbol = self.symbols[symbol_idx]
        
        print(f"Neural Network Prediction:")
        print(f"   - Predicted symbol: '{predicted_symbol}'")
        print(f"   - Confidence: {confidence:.4f} ({confidence*100:.2f}%)")
        print(f"   - Symbol index: {symbol_idx}")
        
        # Show top 3 predictions
        top_3_indices = np.argsort(prediction[0])[-3:][::-1]
        print(f"   - Top 3 predictions:")
        for i, idx in enumerate(top_3_indices):
            symbol = self.symbols[idx]
            conf = prediction[0][idx]
            print(f"     {i+1}. '{symbol}' ({conf:.4f})")
        
        return predicted_symbol, confidence

  # Main Application Integration

In [ ]:
import sympy as sp
from sympy import symbols, Eq, solve, simplify, expand, factor
import re

class SymbolicMathSolver:
    def __init__(self):
        self.x, self.y, self.z = symbols('x y z')
        self.common_vars = [self.x, self.y, self.z]
        print(f"SymbolicMathSolver initialized")
        print(f"   - Available variables: x, y, z")
        
    def parse_equation_string(self, equation_str):
        """
        Parse recognized symbols into a valid mathematical expression
        """
        print(f"\nParsing equation string: '{equation_str}'")
        original_str = equation_str
        
        # Clean up the string
        equation_str = equation_str.replace(' ', '')
        print(f"After removing spaces: '{equation_str}'")
        
        # Handle implicit multiplication (e.g., 2x -> 2*x)
        equation_str = re.sub(r'(\d)([a-zA-Z])', r'\1*\2', equation_str)
        print(f"After adding multiplication (digit+letter): '{equation_str}'")
        
        equation_str = re.sub(r'([a-zA-Z])(\d)', r'\1*\2', equation_str)
        print(f"After adding multiplication (letter+digit): '{equation_str}'")
        
        equation_str = re.sub(r'([a-zA-Z])([a-zA-Z])', r'\1*\2', equation_str)
        print(f"After adding multiplication (letter+letter): '{equation_str}'")
        
        # Handle exponents (^ to **)
        if '^' in equation_str:
            equation_str = equation_str.replace('^', '**')
            print(f"After converting exponents (^ to **): '{equation_str}'")
        
        print(f"Final parsed equation: '{equation_str}'")
        return equation_str
    
    def solve_equation(self, equation_str, variable='x'):
        """
        Solve algebraic equation step by step
        """
        print(f"\nStarting symbolic equation solving...")
        print(f"   - Input equation: '{equation_str}'")
        print(f"   - Solving for variable: '{variable}'")
        
        try:
            # Parse the equation string
            cleaned_eq = self.parse_equation_string(equation_str)
            
            # Split by equals sign
            if '=' in cleaned_eq:
                left, right = cleaned_eq.split('=')
                print(f"Split equation - Left side: '{left}', Right side: '{right}'")
                
                left_expr = sp.sympify(left)
                right_expr = sp.sympify(right)
                equation = Eq(left_expr, right_expr)
                print(f"Created SymPy equation: {equation}")
            else:
                # Assume equation equals zero
                print(f"No '=' found, assuming equation equals zero")
                expr = sp.sympify(cleaned_eq)
                equation = Eq(expr, 0)
                print(f"Created SymPy equation: {equation}")
            
            # Determine variable to solve for
            var = symbols(variable)
            print(f"Solving for variable: {var}")
            
            # Solve the equation
            print(f"Applying symbolic solver...")
            solution = solve(equation, var)
            print(f"Found solution: {solution}")
            
            # Generate step-by-step solution
            steps = self.generate_solution_steps(equation, var)
            
            return {
                'equation': equation,
                'solution': solution,
                'steps': steps,
                'success': True
            }
            
        except Exception as e:
            print(f"Error in symbolic solving: {str(e)}")
            return {
                'error': str(e),
                'success': False
            }
    
    def generate_solution_steps(self, equation, variable):
        """
        Generate step-by-step solution explanation
        """
        print(f"\nGenerating step-by-step solution...")
        steps = []
        
        # Original equation
        step1 = f"Original equation: {equation}"
        steps.append(step1)
        print(f"   Step 1: {step1}")
        
        # Simplify both sides
        left_simplified = simplify(equation.lhs)
        right_simplified = simplify(equation.rhs)
        
        if left_simplified != equation.lhs or right_simplified != equation.rhs:
            step2 = f"Simplified: {Eq(left_simplified, right_simplified)}"
            steps.append(step2)
            print(f"   Step 2: {step2}")
        
        # Move terms to one side
        moved_eq = Eq(left_simplified - right_simplified, 0)
        step3 = f"Move all terms to left: {moved_eq}"
        steps.append(step3)
        print(f"   Step 3: {step3}")
        
        # Solve
        solution = solve(equation, variable)
        step4 = f"Solution: {variable} = {solution}"
        steps.append(step4)
        print(f"   Step 4: {step4}")
        
        print(f"Generated {len(steps)} solution steps")
        return steps
    
    def validate_solution(self, original_eq, solution, variable):
        """
        Validate the solution by substituting back
        """
        print(f"\nValidating solution...")
        print(f"   - Original equation: {original_eq}")
        print(f"   - Solution to check: {variable} = {solution}")
        
        try:
            var = symbols(variable)
            if solution:  # If solution exists
                test_value = solution[0] if isinstance(solution, list) else solution
                substituted = original_eq.subs(var, test_value)
                simplified_result = substituted.simplify()
                is_valid = simplified_result == 0 or simplified_result == True
                
                print(f"   - Substituted equation: {substituted}")
                print(f"   - Simplified result: {simplified_result}")
                print(f"   - Validation result: {'VALID' if is_valid else 'INVALID'}")
                
                return is_valid
            else:
                print(f"   - No solution to validate")
                return False
        except Exception as e:
            print(f"   - Error during validation: {e}")
            return False

# Main Application Integration

In [ ]:
import cv2
import numpy as np
# Note: These classes are defined in previous cells
# from neural_recognizer import MathSymbolCNN
# from symbolic_solver import SymbolicMathSolver 
# from data_preprocessing import HandwritingPreprocessor

class MathTutor:
    def __init__(self):
        self.neural_model = MathSymbolCNN()
        self.symbolic_solver = SymbolicMathSolver()
        self.preprocessor = HandwritingPreprocessor()
        
    def process_handwritten_equation(self, image_path):
        """
        Complete pipeline: Image → Recognition → Solving
        """
        try:
            # Step 1: Segment the equation image
            segments = self.preprocessor.segment_equation(image_path)
            
            # Step 2: Recognize each symbol
            recognized_symbols = []
            for segment in segments:
                # Preprocess segment for neural network
                processed_img = self._prepare_segment(segment['image'])
                
                # Recognize symbol
                symbol, confidence = self.neural_model.predict_symbol(processed_img)
                
                recognized_symbols.append({
                    'symbol': symbol,
                    'confidence': confidence,
                    'position': segment['position']
                })
            
            # Step 3: Construct equation string
            equation_string = self._construct_equation(recognized_symbols)
            
            # Step 4: Solve symbolically
            solution_result = self.symbolic_solver.solve_equation(equation_string)
            
            return {
                'recognized_equation': equation_string,
                'symbols': recognized_symbols,
                'solution': solution_result,
                'success': True
            }
            
        except Exception as e:
            return {
                'error': str(e),
                'success': False
            }
    
    def _prepare_segment(self, segment_image):
        """
        Prepare individual segment for neural network prediction
        """
        # Resize to model input size
        resized = cv2.resize(segment_image, (128, 128))
        
        # Normalize
        normalized = resized.astype('float32') / 255.0
        
        # Reshape for model
        return normalized.reshape(1, 128, 128, 1)
    
    def _construct_equation(self, recognized_symbols):
        """
        Construct equation string from recognized symbols
        """
        # Sort by position (left to right)
        symbols = sorted(recognized_symbols, key=lambda s: s['position'][0])
        
        # Build equation string
        equation = ''.join([s['symbol'] for s in symbols])
        
        return equation
    
    def provide_tutoring_feedback(self, student_solution, correct_solution):
        """
        Compare student's work with correct solution and provide feedback
        """
        feedback = []
        
        if student_solution == correct_solution:
            feedback.append("✓ Excellent! Your solution is correct.")
        else:
            feedback.append("✗ The solution needs correction.")
            feedback.append(f"Your answer: {student_solution}")
            feedback.append(f"Correct answer: {correct_solution}")
            
        return feedback

# Example usage and testing
if __name__ == "__main__":
    tutor = MathTutor()
    
    # Process an equation image
    result = tutor.process_handwritten_equation("sample_equation.png")
    
    if result['success']:
        print(f"Recognized equation: {result['recognized_equation']}")
        print(f"Solution: {result['solution']['solution']}")
        print("\nStep-by-step solution:")
        for step in result['solution']['steps']:
            print(f"  {step}")

# TESTING NEUROSYMBOLIC AI MATH TUTOR

In [ ]:
print("="*70)
print(" COMPREHENSIVE SYSTEM TEST - AI MATH TUTOR ")
print("="*70)

print("\nPHASE 1: INDIVIDUAL COMPONENT TESTING")
print("-" * 50)

# Test 1: HandwritingPreprocessor
print("\n1. Testing HandwritingPreprocessor...")
try:
    preprocessor = HandwritingPreprocessor()
    print(f"HandwritingPreprocessor created successfully")
    print(f"   - Image size setting: {preprocessor.img_size}")
except Exception as e:
    print(f"Error: {e}")

# Test 2: MathSymbolCNN
print("\n2. Testing MathSymbolCNN...")
try:
    cnn = MathSymbolCNN()
    print(f"MathSymbolCNN created successfully")
    print(f"   - Number of symbol classes: {cnn.num_classes}")
    print(f"   - Symbol set: {cnn.symbols}")
except Exception as e:
    print(f"Error: {e}")

# Test 3: SymbolicMathSolver
print("\n3. Testing SymbolicMathSolver...")
try:
    solver = SymbolicMathSolver()
    print(f"SymbolicMathSolver created successfully")
    
    # Test equation parsing and solving
    test_equations = ["2*x+3=7", "x^2-4=0", "3*y-6=9"]
    
    for eq in test_equations:
        print(f"\n   Testing equation: {eq}")
        result = solver.solve_equation(eq)
        if result['success']:
            print(f"   Solved! Solution: {result['solution']}")
        else:
            print(f"   Failed: {result['error']}")
            
except Exception as e:
    print(f"Error: {e}")

print("\nPHASE 2: INTEGRATED SYSTEM TESTING")
print("-" * 50)

# Test 4: MathTutor Integration
print("\n4. Testing MathTutor Integration...")
try:
    tutor = MathTutor()
    print("MathTutor created successfully")
    print("All neurosymbolic components integrated properly")
except Exception as e:
    print(f"Error creating MathTutor: {e}")

print("\nPHASE 3: FULL PIPELINE TEST")
print("-" * 50)

# Test 5: Full Pipeline with Available Image
print("\n5. Testing Full Neurosymbolic Pipeline...")
try:
    image_path = "data/1st.png"
    print(f"Using test image: {image_path}")
    
    # Check if image exists
    import os
    if os.path.exists(image_path):
        print(f"Test image found")
        
        # Run full pipeline
        result = tutor.process_handwritten_equation(image_path)
        
        print(f"\nPIPELINE TEST RESULTS:")
        if result['success']:
            print(f"Pipeline completed successfully!")
            print(f"   - Equation recognized: '{result['recognized_equation']}'")
            print(f"   - Symbols detected: {len(result['symbols'])}")
            if result['solution']['success']:
                print(f"   - Solution found: {result['solution']['solution']}")
            else:
                print(f"   - Solution error: {result['solution']['error']}")
        else:
            print(f"Pipeline failed: {result['error']}")
    else:
        print(f"Test image not found: {image_path}")
        print(f"   Pipeline test skipped - create a test image to run full demo")
        
except Exception as e:
    print(f"Error in pipeline test: {e}")

print(f"\n" + "="*70)
print(" NEUROSYMBOLIC SYSTEM TEST COMPLETE ")
print("="*70)

print(f"\nSYSTEM SUMMARY:")
print(f"   Neural Component: Symbol Recognition CNN")
print(f"   Symbolic Component: SymPy-based Equation Solver") 
print(f"   Integration: Seamless neurosymbolic pipeline")
print(f"   Purpose: Handwritten equation recognition + algebraic solving")
print(f"\nNext Steps:")
print(f"   1. Train the neural network with handwritten symbol dataset")
print(f"   2. Test with various handwritten equation images")  
print(f"   3. Add more sophisticated symbolic reasoning rules")
print(f"   4. Implement educational feedback mechanisms")

# Demo with Your Equation Images

In [ ]:
# Automatically detect and process equation images from data folder
import os
import cv2
import numpy as np
from glob import glob

def scan_equation_images(data_folder="data"):
    """
    Scan the data folder for image files that can be processed
    """
    print(f"Scanning for equation images in: {data_folder}")
    
    # Common image extensions
    image_extensions = ['*.png', '*.jpg', '*.jpeg', '*.bmp', '*.tiff', '*.tif']
    
    image_files = []
    for ext in image_extensions:
        pattern = os.path.join(data_folder, ext)
        image_files.extend(glob(pattern))
    
    print(f"Found {len(image_files)} image files:")
    for i, img_path in enumerate(image_files, 1):
        file_size = os.path.getsize(img_path) if os.path.exists(img_path) else 0
        print(f"  {i}. {os.path.basename(img_path)} ({file_size} bytes)")
    
    return image_files

def display_image_info(image_path):
    """
    Display basic information about an image
    """
    if os.path.exists(image_path):
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            print(f"\nImage Info: {os.path.basename(image_path)}")
            print(f"   - Shape: {img.shape} (height x width)")
            print(f"   - Data type: {img.dtype}")
            print(f"   - Min pixel value: {img.min()}")
            print(f"   - Max pixel value: {img.max()}")
            print(f"   - File size: {os.path.getsize(image_path)} bytes")
            return True
        else:
            print(f"   - Error: Could not load image {image_path}")
            return False
    else:
        print(f"   - Error: File not found {image_path}")
        return False

# Check if data folder exists
data_folder = "data"
if not os.path.exists(data_folder):
    print(f"Creating data folder: {data_folder}")
    os.makedirs(data_folder)
    print("Please place your equation image files in the 'data' folder")
    print("Supported formats: PNG, JPG, JPEG, BMP, TIFF")
else:
    print(f"Data folder exists: {data_folder}")

# Scan for available images
available_images = scan_equation_images(data_folder)

if available_images:
    print(f"\nReady to process {len(available_images)} equation images!")
    print("You can now run the full demo in the next cell.")
    
    # Display info for each image
    for img_path in available_images:
        display_image_info(img_path)
else:
    print(f"\nNo images found in {data_folder} folder.")
    print("Please add your handwritten equation images to the data folder:")
    print("  - Supported formats: PNG, JPG, JPEG, BMP, TIFF") 
    print("  - Place images with equations like '2x+3=7', 'x^2-4=0', etc.")
    print("  - Then run the demo cell to see the neurosymbolic system in action!")

# Run Full Neurosymbolic Demo

In [ ]:
# Run the complete neurosymbolic pipeline with your equation images
print("NEUROSYMBOLIC AI MATH TUTOR - FULL DEMO")
print("=" * 60)

# Initialize the AI Math Tutor
print("\nInitializing AI Math Tutor...")
tutor = MathTutor()

# Get all available images from data folder
available_images = scan_equation_images("data")

if not available_images:
    print("\nNo images found in data folder!")
    print("Please add your handwritten equation images to the 'data' folder and try again.")
else:
    print(f"\nProcessing {len(available_images)} equation images...")
    
    # Process each image found in the data folder
    for i, image_path in enumerate(available_images, 1):
        print(f"\n" + "=" * 60)
        print(f"DEMO {i}/{len(available_images)}: Processing {os.path.basename(image_path)}")
        print("=" * 60)
        
        # Display image information first
        display_image_info(image_path)
        
        try:
            # Run the complete neurosymbolic pipeline
            print(f"\nRunning neurosymbolic pipeline on {os.path.basename(image_path)}...")
            result = tutor.process_handwritten_equation(image_path)
            
            print(f"\nFINAL RESULTS FOR {os.path.basename(image_path)}:")
            print("-" * 40)
            
            if result['success']:
                print(f"SUCCESS! Neurosymbolic pipeline completed")
                print(f"Recognized equation: '{result['recognized_equation']}'")
                print(f"Number of symbols detected: {len(result['symbols'])}")
                
                # Show individual symbol results
                if result['symbols']:
                    print(f"\nDetailed symbol recognition:")
                    for j, sym in enumerate(result['symbols'], 1):
                        print(f"  Symbol {j}: '{sym['symbol']}' (confidence: {sym['confidence']:.3f}) at position {sym['position']}")
                
                # Show mathematical solution
                if result['solution']['success']:
                    print(f"\nMathematical Solution:")
                    print(f"Solution: {result['solution']['solution']}")
                    print(f"\nStep-by-step solving process:")
                    for k, step in enumerate(result['solution']['steps'], 1):
                        print(f"  Step {k}: {step}")
                        
                    # Validate the solution
                    if result['solution']['solution']:
                        original_eq = result['solution']['equation']
                        solution_val = result['solution']['solution']
                        is_valid = tutor.symbolic_solver.validate_solution(original_eq, solution_val, 'x')
                        print(f"\nSolution validation: {'PASSED' if is_valid else 'FAILED'}")
                else:
                    print(f"\nMathematical solving failed: {result['solution']['error']}")
                    
            else:
                print(f"FAILED: {result['error']}")
                
        except Exception as e:
            print(f"ERROR during processing: {str(e)}")
            import traceback
            print("Full error traceback:")
            traceback.print_exc()

    print(f"\n" + "=" * 60)
    print("NEUROSYMBOLIC DEMO COMPLETED")
    print("=" * 60)

    print(f"\nDEMO SUMMARY:")
    print(f"Processed {len(available_images)} equation images")
    print(f"This demo showcased the complete neurosymbolic AI system:")
    print(f"1. NEURAL COMPONENT: Image preprocessing and symbol recognition")
    print(f"2. SYMBOLIC COMPONENT: Equation parsing and algebraic solving") 
    print(f"3. INTEGRATION: Seamless pipeline from handwritten images to solutions")
    print(f"\nNOTE: The neural network is not trained, so symbol recognition is simulated.")
    print(f"For a complete system, you would need to:")
    print(f"- Train the CNN on a large dataset of handwritten math symbols")
    print(f"- Add more sophisticated image preprocessing")
    print(f"- Extend the symbolic solver for more complex equations")
    print(f"- Implement educational feedback and tutoring features")